In [8]:
import geopandas as gpd
import os

In [3]:
def create_land_buffer(
    country_gdf, harbor_gdf, buffer_country, buffer_harbor
):
    """
    Creates a buffer around harbors and countries and saves it GeoJSON.
    Arguments:
        country_gdf: geodataframe
        harbor_gdf: geodataframe
        buffer_harbor: size buffer around harbors (km)
        buffer_country: size buffer around countries (km)
    Returns:
        None, but saves GeoJSON
    """
    # conversion https://www.usna.edu/Users/oceano/pguth/md_help/html/approx_equivalents.htm
    # 1 degree = 111 km
    # 1 km = 0.009 degrees
    # This is usually accurate enough, but should not be used for anything near the poles
    buffer_country_deg = buffer_country * 0.009
    buffer_harbor_deg = buffer_harbor * 0.009
    # Dissolve it
    countries_dissolved = gpd.GeoDataFrame(country_gdf.dissolve()["geometry"])
    harbors_dissolved = gpd.GeoDataFrame(harbor_gdf.dissolve()["geometry"])
    # Create the buffers
    buffered_harbors = gpd.GeoDataFrame(harbors_dissolved.buffer(buffer_harbor_deg))
    buffered_harbors.columns = ["geometry"]
    buffered_countries = gpd.GeoDataFrame(
        countries_dissolved.buffer(buffer_country_deg)
    )
    buffered_countries.columns = ["geometry"]
    # Combine the two bufferes
    buffer_both = buffered_countries.union(buffered_harbors)
    # Substract the countries from the buffer
    buffer_diff = gpd.GeoDataFrame(buffer_both.difference(countries_dissolved))
    buffer_diff.columns = ["geometry"]
    buffer_diff = buffer_diff.dissolve()
    buffer_diff.to_file(
        "results"
        + os.sep
        + "harbor_{}km_coast_{}km_buffer.geojson".format(buffer_harbor, buffer_country),
        driver="GeoJSON",
    )

In [6]:
harbors = gpd.read_file("global_harbors.json")
harbors.head()

,id,portname,code,prttype,prtsize,status,maxdepth,maxlength,annualcapacitymt,humuse,...,country,lastcheckdate,remarks,url_lca,source,createdate,updatedate,geonameid,gdb_geomattr_data,geometry
0,wld_trs_ports_wfp.14314,Watsi-Genge,None,River,Very Small,Unknown,NaN,NaN,NaN,Unknown,...,Democratic Republic of the Congo,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,204280,None,POINT (20.62966 -0.94560)
1,wld_trs_ports_wfp.14315,Charlotte (Skidegate),CASKI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,Canada,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,6148858,None,POINT (-132.00969 53.24742)
2,wld_trs_ports_wfp.14316,Homer,USHOI,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,None,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,5864145,None,POINT (-151.41553 59.60159)
3,wld_trs_ports_wfp.14317,Hartsdale,USHAS,Sea,Unknown,Open,NaN,NaN,NaN,Unknown,...,None,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,5120141,None,POINT (-74.00210 40.70610)
4,wld_trs_ports_wfp.14318,Killingholme,GBKGH,Sea,Unknown,Unknown,NaN,NaN,NaN,Unknown,...,None,NaT,None,None,None,2021-02-24 11:52:47.493000+00:00,2021-02-24 11:52:47.493000+00:00,2641323,None,POINT (-0.21512 53.64561)


In [10]:
countries = gpd.read_file("Countries" + os.sep + "ne_50m_admin_0_countries.shp")
countries.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,1,3,Zimbabwe,ZWE,0,2,Sovereign country,1,Zimbabwe,...,None,None,None,None,None,None,None,None,None,"POLYGON ((31.28789 -22.40205, 31.19727 -22.344..."
1,Admin-0 country,1,3,Zambia,ZMB,0,2,Sovereign country,1,Zambia,...,None,None,None,None,None,None,None,None,None,"POLYGON ((30.39609 -15.64307, 30.25068 -15.643..."
2,Admin-0 country,1,3,Yemen,YEM,0,2,Sovereign country,1,Yemen,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((53.08564 16.64839, 52.58145 16..."
3,Admin-0 country,3,2,Vietnam,VNM,0,2,Sovereign country,1,Vietnam,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((104.06396 10.39082, 104.08301 ..."
4,Admin-0 country,5,3,Venezuela,VEN,0,2,Sovereign country,1,Venezuela,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-60.82119 9.13838, -60.94141 9..."


In [11]:
create_seaweed_land_buffer(countries, harbors, 2.5, 50)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U28'), dtype('float64')) -> None